In [23]:
import pandas as pd
from datetime import datetime
from IPython.display import display, HTML

In [33]:
# Load the datasets
event_data_path = './data/event_data_sherdog.csv'
fighter_info_path = './data/fighter_info.csv'

event_df = pd.read_csv(event_data_path)
fighter_info_df = pd.read_csv(fighter_info_path)

# Remove rows with invalid birth dates
fighter_info_df = fighter_info_df[fighter_info_df['Birth Date'] != '-']

# Parse the dates
event_df['Event Date'] = pd.to_datetime(event_df['Event Date'], errors='coerce', utc=True).dt.tz_convert(None)
fighter_info_df['Birth Date'] = pd.to_datetime(fighter_info_df['Birth Date'], errors='coerce', utc=True)

# Drop duplicate columns
fighter_info_df = fighter_info_df.drop(columns=['Weight Class'])

# Merge the datasets on Fighter 1 and Fighter 2
merged_df1 = pd.merge(event_df, fighter_info_df, left_on='Fighter 1 ID', right_on='Fighter_ID', how='left', suffixes=('', '_F1'))
merged_df2 = pd.merge(merged_df1, fighter_info_df, left_on='Fighter 2 ID', right_on='Fighter_ID', how='left', suffixes=('_F1', '_F2'))

# Calculate ages at the time of the fight
merged_df2['Age_F1'] = merged_df2.apply(lambda row: row['Event Date'].year - row['Birth Date_F1'].year - ((row['Event Date'].month, row['Event Date'].day) < (row['Birth Date_F1'].month, row['Birth Date_F1'].day)), axis=1)
merged_df2['Age_F2'] = merged_df2.apply(lambda row: row['Event Date'].year - row['Birth Date_F2'].year - ((row['Event Date'].month, row['Event Date'].day) < (row['Birth Date_F2'].month, row['Birth Date_F2'].day)), axis=1)

# Filter for fighters who are 40 years or older
fighters_40_or_older = merged_df2[(merged_df2['Age_F1'] >= 40) | (merged_df2['Age_F2'] >= 40)]

# Initialize dictionaries for win/loss counts
fighter_wins = {}
fighter_losses = {}

# Update win/loss records based on fight outcomes
for _, row in fighters_40_or_older.iterrows():
    if row['Age_F1'] >= 40:
        fighter1 = row['Fighter 1']
        if fighter1 not in fighter_wins:
            fighter_wins[fighter1] = 0
            fighter_losses[fighter1] = 0
        if row['Winning Fighter'].lower() == fighter1.lower():
            fighter_wins[fighter1] += 1
        else:
            fighter_losses[fighter1] += 1
    
    if row['Age_F2'] >= 40:
        fighter2 = row['Fighter 2']
        if fighter2 not in fighter_wins:
            fighter_wins[fighter2] = 0
            fighter_losses[fighter2] = 0
        if row['Winning Fighter'].lower() == fighter2.lower():
            fighter_wins[fighter2] += 1
        else:
            fighter_losses[fighter2] += 1

# Calculate the total wins and losses for fighters 40 years or older
total_wins = sum(fighter_wins.values())
total_losses = sum(fighter_losses.values())

# print("Total Wins:", total_wins)
# print("Total Losses:", total_losses)

# Categorize the methods for simplicity without a function
win_methods_df['Category'] = win_methods_df['Winning Method'].apply(
    lambda method: 'KO/TKO' if 'KO' in method or 'TKO' in method else 
    'Submission' if 'Submission' in method or 'Technical Submission' in method else 
    'Decision' if 'Decision' in method or 'Technical Decision' in method else 
    'Other'
)

loss_methods_df['Category'] = loss_methods_df['Losing Method'].apply(
    lambda method: 'KO/TKO' if 'KO' in method or 'TKO' in method else 
    'Submission' if 'Submission' in method or 'Technical Submission' in method else 
    'Decision' if 'Decision' in method or 'Technical Decision' in method else 
    'Other'
)

# Calculate the sum of each category in wins and losses
win_category_counts = win_methods_df['Category'].value_counts()
loss_category_counts = loss_methods_df['Category'].value_counts()

# Combine results
total_wins = sum(fighter_wins.values())
total_losses = sum(fighter_losses.values())

win_summary = f" Wins: {total_wins}\n"
for category, count in win_category_counts.items():
    win_summary += f"- {category}: {count}\n"

loss_summary = f"Losses: {total_losses}\n"
for category, count in loss_category_counts.items():
    loss_summary += f"- {category}: {count}\n"

print(win_summary)
print(loss_summary)

 Wins: 71
- Decision: 27
- KO/TKO: 27
- Submission: 17

Losses: 131
- KO/TKO: 60
- Decision: 47
- Submission: 21
- Other: 3



In [27]:
# Specific winning methods

win_methods = []
loss_methods = []

for _, row in fighters_40_or_older.iterrows():
    if row['Age_F1'] >= 40:
        if row['Winning Fighter'].lower() == row['Fighter 1'].lower():
            win_methods.append(row['Winning Method'])
        else:
            loss_methods.append(row['Winning Method'])
    
    if row['Age_F2'] >= 40:
        if row['Winning Fighter'].lower() == row['Fighter 2'].lower():
            win_methods.append(row['Winning Method'])
        else:
            loss_methods.append(row['Winning Method'])

win_methods_df = pd.DataFrame(win_methods, columns=['Winning Method'])
loss_methods_df = pd.DataFrame(loss_methods, columns=['Losing Method'])
win_method_counts = win_methods_df['Winning Method'].value_counts()
loss_method_counts = loss_methods_df['Losing Method'].value_counts()

print("Winning Methods:")
print(win_method_counts)
print("\nLosing Methods:")
print(loss_method_counts)


Winning Methods:
Winning Method
Decision (Unanimous)                       20
TKO (Punches)                               8
Submission (Rear-Naked Choke)               7
Decision (Split)                            6
KO (Punches)                                6
KO (Punch)                                  5
Submission (Armbar)                         3
TKO (Elbows and Punches)                    2
Submission (Neck Crank)                     1
KO (Knee)                                   1
Submission (Anaconda Choke)                 1
Submission (Arm-Triangle Choke)             1
TKO (Elbows and Knees)                      1
KO (Elbow)                                  1
Decision (Majority)                         1
Submission (Ezekiel Choke)                  1
KO (Knee to the Body)                       1
Technical Submission (Rear-Naked Choke)     1
Submission (Face Crank)                     1
TKO (Punch)                                 1
Submission (Scarf Hold)                     1
TK

In [25]:
first_10_fights = fighters_40_or_older.head(10)
data_subset = first_10_fights[['Event Name', 'Event Date', 'Fighter 1', 'Age_F1', 'Fighter 2', 'Age_F2', 'Winning Fighter', 'Winning Method', 'Weight Class', 'Fight Type']]

styled_table = data_subset.style.set_table_styles(
    [
        {'selector': 'thead th', 'props': [('background-color', '#007BFF'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#f2f2f2')]},
        {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#ffffff')]},
        {'selector': 'tbody td', 'props': [('padding', '8px'), ('text-align', 'left'), ('border', '1px solid #ddd')]},
        {'selector': 'thead th', 'props': [('padding', '10px'), ('text-align', 'left'), ('border', '1px solid #ddd')]}
    ]
).set_properties(
    **{'background-color': '#fff', 'color': '#333', 'border-color': '#ddd'}
).set_caption("Fights Involving Fighters Over 40 Years Old")

display(styled_table)

,Event Name,Event Date,Fighter 1,Age_F1,Fighter 2,Age_F2,Winning Fighter,Winning Method,Weight Class,Fight Type
54,UFC on ESPN 57 - Cannonier vs. Imavov,2024-06-08 07:00:00,Nassourdine Imavov,29.000000,Jared Cannonier,40.000000,Nassourdine Imavov,TKO (Punches),Middleweight,Main Event
161,UFC Fight Night 239 - Tuivasa vs. Tybura,2024-03-16 07:00:00,Ovince St. Preux,40.000000,Kennedy Nzechukwu,31.000000,Ovince St. Preux,Decision (Split),Light Heavyweight,Undercard
199,UFC 296 - Edwards vs. Covington,2023-12-16 08:00:00,Shavkat Rakhmonov,29.000000,Stephen Thompson,40.000000,Shavkat Rakhmonov,Submission (Rear-Naked Choke),Welterweight,Undercard
225,UFC on ESPN 52 - Dariush vs. Tsarukyan,2023-12-02 08:00:00,Joaquim Silva,34.000000,Clay Guida,41.000000,Joaquim Silva,Decision (Unanimous),Lightweight,Undercard
270,UFC 301 - Pantoja vs. Erceg,2024-05-04 07:00:00,Ismael Bonfim,28.000000,Vinc Pichel,41.000000,Ismael Bonfim,Decision (Unanimous),Lightweight,Undercard
327,UFC 300 - Pereira vs. Hill,2024-04-13 07:00:00,Kayla Harrison,nan,Hollys Daughter' Holm,42.000000,Kayla Harrison,Submission (Rear-Naked Choke),Bantamweight,Undercard
331,UFC 300 - Pereira vs. Hill,2024-04-13 07:00:00,Bobby Green,37.000000,Jim Miller,40.000000,Bobby Green,Decision (Unanimous),Lightweight,Undercard
338,UFC on ESPN 55 - Nicolau vs. Perez,2024-04-27 07:00:00,Uros Medic,31.000000,Tim Means,40.000000,Uros Medic,TKO (Punches),Welterweight,Undercard
380,UFC 291 - Poirier vs. Gaethje 2,2023-07-29 07:00:00,Alex Pereira,36.000000,Jan Blachowicz,40.000000,Alex Pereira,Decision (Split),Light Heavyweight,Undercard
477,UFC 290 - Volkanovski vs. Rodriguez,2023-07-08 07:00:00,Robbie Lawler,41.000000,Niko Price,33.000000,Robbie Lawler,KO (Punches),Welterweight,Undercard
